## Parallelization - Distribute independent subtasks across multiple LLMs for concurrent processing

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

/Users/ips/Documents/trail/AI Agents/aiagent/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from concurrent.futures import ThreadPoolExecutor # this is abstract representation of pool of threads which can be used to execute tasks in parallel

#list of messagee to translate
a = [
    "Hi ram how are you ?",
    "I am good bro missed you so much",
    "You are so looking good from last time"
]

# Function to process a single message
def process_message(message):
    """
    This function takes the message and translates it to Spanish using the LLM
    """
    return llm.invoke(f"Translate this to spanish:\n{message}.Only give me the translation no explanation or any other text")

# Using ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor(max_workers=3) as executor:
    results = list(executor.map(process_message,a))

# Printing Results
for result in results:
    print(result.content)

Hola Ram, ¿cómo estás?
Estoy bien, hermano. Te extrañé mucho.
Te ves muy bien desde la última vez.


## Routing - Dynamically selects specialized LLM paths based on input characteristics

In [3]:
options = """ 'Life insurance','Health insurance', 'Car insurance', 'Home insurance', 'General Consultation' """
print(options.split(","))
input_question = "My car is damaged and I want to buy a car insurance policy ?"

selection_pompt = f"""
user query: {input_question}
Analyze the user query and select the most relevant option from the list of options {options.split(",")}.
First explain your reasoning and then give me the answer in this XML format:

<reasoning>
Explain your reasoning here
</reasoning>
<answer>
Answer here
</answer>
"""

result = llm.invoke(selection_pompt)

[" 'Life insurance'", "'Health insurance'", " 'Car insurance'", " 'Home insurance'", " 'General Consultation' "]


In [7]:
def extract_option(result):
    return result.content.split("<answer>")[1].split("</answer>")[0].strip()

def extract_reasoning(result):
    return result.content.split("<reasoning>")[1].split("</reasoning>")[0].strip()

option = extract_option(result)
print(option)
print(extract_reasoning(result))

Car insurance
The user is looking for insurance because their car is damaged. The query explicitly mentions "car insurance policy". Therefore, the most relevant option is car insurance.


In [9]:
option

'Car insurance'

In [12]:
# Based on option selected print the tool free number 
if option == 'Car insurance':
    print("Car insurance tool free number is 1800-976-7654")
elif option == 'Home insurance':
    print("Home insurance tool free number is 1800-976-7654")

Car insurance tool free number is 1800-976-7654
